## Importing Dependencies

In [1]:
import requests
from bs4 import BeautifulSoup
import ccxt
from tqdm.notebook import tqdm
import pandas as pd

## Extracting 50 trading pairs with highest market cap on Binance

In [2]:
url = 'https://coinmarketcap.com/exchanges/binance/'
response = requests.get(url)
content = response.text

In [3]:
soup = BeautifulSoup(content, 'html.parser')

In [4]:
market_data_html_class = 'sc-7b3ac367-3 NFswn cmc-table'
table = soup.find(class_=market_data_html_class)

In [5]:
column_index = 2
trading_pairs = []

for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > column_index:
        trading_pairs.append(cells[column_index].text)

for i in range(10):
    print(trading_pairs[i])

BTC/FDUSD
ETH/FDUSD
BTC/USDT
ETH/USDT
SOL/USDT
USDC/USDT
FDUSD/USDT
CATI/USDT
SOL/FDUSD
NEIRO/USDT


## Extracting OHLCV data for each trading pair

In [6]:
binance = ccxt.binance()

In [7]:
def fetch_ohlcv(trading_pair, interval, start_ts):
    from_ts = binance.parse8601(start_ts)
    ohlcv = binance.fetch_ohlcv(trading_pair, interval, since=from_ts, limit=1000)
    while True:
        from_ts = ohlcv[-1][0] + 1
        new_ohlcv = binance.fetch_ohlcv(trading_pair, interval, since=from_ts, limit=1000)
        ohlcv.extend(new_ohlcv)
        if len(new_ohlcv) != 1000:
            break
    data_dict = {trading_pair: ohlcv}
    return data_dict

In [8]:
start_ts = '2024-06-21 19:15:00'
interval = '15m'
full_ohlcv = {}

for trading_pair in tqdm(trading_pairs, desc='Fetching OHLCV Data', unit='Trading Pair'):
    ohlcv = fetch_ohlcv(trading_pair, interval, start_ts)
    full_ohlcv.update(ohlcv)

Fetching OHLCV Data:   0%|          | 0/50 [00:00<?, ?Trading Pair/s]

In [14]:
data_for_df = []

for trading_pair, ohlcv in full_ohlcv.items():
    for entry in ohlcv:
        data_for_df.append([entry[0], trading_pair] + entry[1:])

In [15]:
df = pd.DataFrame(data_for_df, columns=['Timestamp', 'Trading Pair', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df.set_index('Timestamp', inplace=True)
df.head()

,Trading Pair,Open,High,Low,Close,Volume
Timestamp,,,,,,
2024-06-21 19:15:00,BTC/FDUSD,63942.02,64072.13,63933.64,64004.01,319.60860
2024-06-21 19:30:00,BTC/FDUSD,64002.94,64133.57,63998.18,64115.95,318.18427
2024-06-21 19:45:00,BTC/FDUSD,64114.12,64240.00,64052.89,64197.99,371.73904
2024-06-21 20:00:00,BTC/FDUSD,64196.03,64374.00,64169.85,64249.50,409.04486
2024-06-21 20:15:00,BTC/FDUSD,64251.91,64435.48,64232.43,64235.30,271.01192


## Calculating 24-Hour trading volume

In [16]:
df['24-Hour Volume'] = df['Volume'].rolling(window=96).sum()

In [13]:
df['24-Hour Volume'] = df['24-Hour Volume'].fillna(0)
df

,Trading Pair,Open,High,Low,Close,Volume,24-Hour Volume
Timestamp,,,,,,,
2024-06-21 19:15:00,BTC/FDUSD,63942.0200,64072.1300,63933.6400,64004.0100,319.60860,0.0
2024-06-21 19:30:00,BTC/FDUSD,64002.9400,64133.5700,63998.1800,64115.9500,318.18427,0.0
2024-06-21 19:45:00,BTC/FDUSD,64114.1200,64240.0000,64052.8900,64197.9900,371.73904,0.0
2024-06-21 20:00:00,BTC/FDUSD,64196.0300,64374.0000,64169.8500,64249.5000,409.04486,0.0
2024-06-21 20:15:00,BTC/FDUSD,64251.9100,64435.4800,64232.4300,64235.3000,271.01192,0.0
...,...,...,...,...,...,...,...
2024-09-21 21:15:00,FIDA/USDT,0.3205,0.3205,0.3174,0.3187,430107.40000,67507804.7
2024-09-21 21:30:00,FIDA/USDT,0.3186,0.3205,0.3179,0.3189,389173.20000,67561959.6
2024-09-21 21:45:00,FIDA/USDT,0.3188,0.3196,0.3185,0.3191,58445.40000,67301593.9


## Fetching Order Book data

In [39]:
#TODO: Fetch and combine Order Book Data

## Notes

- Some trading pairs, being new, have lesser OHLCV data.

- Used 24h summed up trading volume, where first day might have 0 as trading volume